# Spam - Ham Classification

In [42]:
import pandas as pd
# Leer el archivo de texto "data.csv" en un DataFrame de pandas
df = pd.read_csv("data.csv", sep=',', header=None, names=['label', 'message'])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [43]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# Convertir a minúsculas
df['message'] = df['message'].str.lower()

# Eliminar números
df['message'] = df['message'].str.replace(r'\d+', '')

# Eliminar signos de puntuación
df['message'] = df['message'].str.translate(str.maketrans('', '', string.punctuation))

# Eliminar palabras vacías (stop words)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['message'] = df['message'].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

# Realizar lematización o stemming (opcional)
stemmer = PorterStemmer()
df['message'] = df['message'].apply(lambda x: ' '.join(stemmer.stem(word) for word in x.split()))

# Codificar las etiquetas en números
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

# Mostrar el DataFrame
df.head()

C:\Users\carev\AppData\Local\Temp\ipykernel_155760\70690099.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['message'] = df['message'].str.replace(r'\d+', '')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,message
0,0,go jurong point crazi avail bugi n great world...
1,0,ok lar joke wif u oni
2,1,free entri wkli comp win fa cup final tkt st m...
3,0,u dun say earli hor u c alreadi say
4,0,nah dont think goe usf live around though


In [44]:
from sklearn.model_selection import train_test_split

# Dividir el DataFrame en conjuntos de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

# X_train: Conjunto de entrenamiento (mensajes)
# y_train: Etiquetas correspondientes al conjunto de entrenamiento (spam o ham)
# X_valid: Conjunto de validación (mensajes)
# y_valid: Etiquetas correspondientes al conjunto de validación

In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Tokenización de texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_valid_sequences = tokenizer.texts_to_sequences(X_valid)

# Padding de secuencias para que tengan la misma longitud
max_length = 100  # longitud máxima de una secuencia
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post', truncating='post')
X_valid_padded = pad_sequences(X_valid_sequences, maxlen=max_length, padding='post', truncating='post')



In [46]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


In [47]:
model = Sequential()


In [48]:

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))


In [49]:

model.add(LSTM(units=64, return_sequences=True))

In [50]:

model.add(Dense(1, activation='sigmoid'))


In [51]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [52]:
epochs = 5
history = model.fit(X_train_padded, y_train, epochs=epochs, validation_data=(X_valid_padded, y_valid))


Epoch 1/5
140/140 [==============================] - 10s 48ms/step - loss: 0.3771 - accuracy: 0.8810 - val_loss: 0.2084 - val_accuracy: 0.9525
Epoch 2/5
140/140 [==============================] - 6s 45ms/step - loss: 0.1119 - accuracy: 0.9766 - val_loss: 0.1310 - val_accuracy: 0.9719
Epoch 3/5
140/140 [==============================] - 7s 48ms/step - loss: 0.0600 - accuracy: 0.9891 - val_loss: 0.1054 - val_accuracy: 0.9778
Epoch 4/5
140/140 [==============================] - 6s 40ms/step - loss: 0.0358 - accuracy: 0.9934 - val_loss: 0.1072 - val_accuracy: 0.9804
Epoch 5/5
140/140 [==============================] - 5s 37ms/step - loss: 0.0305 - accuracy: 0.9944 - val_loss: 0.1327 - val_accuracy: 0.9746


In [53]:
# Función para predecir si un mensaje es "spam" o "no spam"
def predecir_spam_o_no_spam(mensaje):
    # Preprocesar el mensaje de la misma manera que hicimos con los datos de entrenamiento
    mensaje = mensaje.lower()
    mensaje = ''.join([c for c in mensaje if c not in string.punctuation])
    mensaje = ' '.join(word for word in mensaje.split() if word not in stop_words)
    mensaje = ' '.join(stemmer.stem(word) for word in mensaje.split())
    
    # Convertir el mensaje en una secuencia de tokens y aplicar el padding
    mensaje_secuencia = tokenizer.texts_to_sequences([mensaje])
    mensaje_padded = pad_sequences(mensaje_secuencia, maxlen=max_length, padding='post', truncating='post')
    
    # Realizar la predicción
    predicciones = model.predict(mensaje_padded)
    
    # Clasificar el mensaje como "spam" o "no spam" en función de las predicciones
    if predicciones[0][0] >= 0.5:
        return "Spam"
    else:
        return "No Spam (Ham)"


In [54]:
mensaje_prueba = "Congratulations ur awarded $500 of CD vouchers or 125gift guaranteed & Free entry 2 £100 wkly draw txt MUSIC to 87066"
resultado = predecir_spam_o_no_spam(mensaje_prueba)
print(f"Predicción: {resultado}")


1/1 [==============================] - 0s 483ms/step
Predicción: Spam
